In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import shap

import xgboost
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


PATH_PROJECT = "/home/yoshraf/projects/master-analysis-inequality-mobility/"

X_train = pd.read_parquet(f"{PATH_PROJECT}data/discrete_choice/X_train.parquet")
X_test = pd.read_parquet(f"{PATH_PROJECT}data/discrete_choice/X_test.parquet")
y_train = pd.read_parquet(f"{PATH_PROJECT}data/discrete_choice/y_train.parquet")
y_test = pd.read_parquet(f"{PATH_PROJECT}data/discrete_choice/y_test.parquet")

In [2]:
def valid_model(y_train, y_test, y_pred_train, y_pred_test):
    mae_train = mean_absolute_error(y_train, y_pred_train)
    mse_train = mean_squared_error(y_train, y_pred_train)
    r2_train = r2_score(y_train, y_pred_train)
    mae_test = mean_absolute_error(y_test, y_pred_test)
    mse_test = mean_squared_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)
    print("----Train---")
    print(f"MAE: {mae_train:.2f}")
    print(f"MSE: {mse_train:.2f}")
    print(f"R2: {r2_train:.2f}")
    print("---- Test ---")
    print(f"MAE: {mae_test:.2f}")
    print(f"MSE: {mse_test:.2f}")
    print(f"R2: {r2_test:.2f}")
    return None

def lr_modeling(X_train, X_test, y_train, y_test):
    reg = LinearRegression()
    reg.fit(X_train, y_train)
    y_pred_train = reg.predict(X_train)
    y_pred_test = reg.predict(X_test)
    valid_model(y_train, y_test, y_pred_train, y_pred_test)
    return reg

def rf_modeling(X_train, X_test, y_train, y_test, params):
    reg = RandomForestRegressor(**params)
    reg.fit(X_train, y_train)
    y_pred_train = reg.predict(X_train)
    y_pred_test = reg.predict(X_test)
    feats = {}
    for feature, importance in zip(X_train.columns, reg.feature_importances_):
        feats[feature] = importance * 100
    df_imp = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Importance'}).sort_values("Importance", ascending=False)
    print("Main Features:")
    print(df_imp.head(10))
    valid_model(y_train, y_test, y_pred_train, y_pred_test)
    return reg

def xgb_modeling(X_train, X_test, y_train, y_test, params):
    model_xgb = xgboost.XGBClassifier(**params)
    model_xgb.fit(X_train, y_train)
    y_pred_train = model_xgb.predict(X_train)
    y_pred_test = model_xgb.predict(X_test)
#     valid_model(y_train, y_test, y_pred_train, y_pred_test)
    return model_xgb


In [3]:
X_train.sample(5)

,,,,,Idade,Total de viagens da pessoa,Hora Saída,loc_origem_count_parada,loc_origem_count_ilum_std,loc_origem_dist_metro,loc_origem_dist_trem,loc_origem_dist_term,loc_origem_dist_ciclo,loc_origem_ACC_TI_A_E_60M,...,loc_destino_dist_ciclo,loc_destino_ACC_TI_A_E_60M,loc_destino_ACC_TI_A_L_TP_,per Quantidade de automóveis,per Quantidade de motocicletas,per Quantidade de bicicletas,per Renda familiar mensal,dist_od,diff_cota_od,Gênero_Masculino
Identifica pessoa,Zona de domicílio,Data da entrevista,Coordenada X domicílio,Coordenada Y domicílio,,,,,,,,,,,,,,,,,,,,,
03190968104,319.0,03092018,325094.0,7388383.0,28.0,4.0,21.5,16,180,2189.789320,1628.996517,3601.470824,1189.578991,4146092.0,...,562.547760,3597666.0,107.0,0.600000,0.2,0.40,6552.238904,4488.164993,-53.837437,0.0
00371018103,37.0,21032018,331493.0,7397183.0,14.0,2.0,7.0,8,90,857.304084,1182.290590,1021.005370,137.373568,4192644.0,...,476.557436,4276168.0,146.0,0.250000,0.0,0.25,4750.000000,1155.943338,-1.160000,1.0
01260034101,126.0,12092017,327154.0,7402515.0,69.0,2.0,19.5,13,28,7292.820519,3671.315501,3144.178957,673.554281,2132466.0,...,284.109260,3415214.0,19.0,0.500000,0.0,0.25,1237.323229,1446.554527,46.946667,1.0
01880028102,188.0,01082017,351679.0,7398874.0,36.0,1.0,6.0,4,21,1176.017391,830.473178,1202.518559,759.299249,4123100.0,...,1375.521103,1465318.0,15.0,0.250000,0.0,0.00,2011.640101,17475.290241,-27.641333,0.0
00850498103,85.0,09082018,329837.0,7395805.0,27.0,5.0,14.0,39,194,1341.046870,1548.274284,1496.072050,482.802743,4285277.0,...,448.223655,4218810.0,163.0,0.666667,0.0,0.00,3708.518358,3672.014842,54.061377,1.0


In [4]:
y_train

,,,,,Modo Principal
Identifica pessoa,Zona de domicílio,Data da entrevista,Coordenada X domicílio,Coordenada Y domicílio,
00282196101,28.0,23092018,331708.0,7393299.0,Ônibus/micro-ônibus/perua do município de São ...
03200074102,320.0,16042018,324560.0,7387434.0,Passageiro de automóvel
01370004101,137.0,02112017,330750.0,7401259.0,Dirigindo automóvel
01411826101,141.0,02092018,331203.0,7403277.0,A pé
01080003101,108.0,07042018,321845.0,7397275.0,A pé
...,...,...,...,...,...
01570826101,157.0,12042018,340354.0,7403240.0,A pé
02420016101,242.0,26082017,337327.0,7389660.0,Dirigindo automóvel
03251069102,325.0,08082018,320038.0,7385317.0,Dirigindo automóvel


In [5]:
params_xgb = {
    "objective":"multi:softprob",
    "eval_metric": "mlogloss",
#     "objective":"multi:softmax",
    "max_depth": 3,
    "subsample": 0.8,
    "learning_rate": 0.1,
    "n_estimators": 10,
#     "scale_pos_weight": 1,
    "seed": 42}
reg_xgb = xgb_modeling(X_train, X_test, y_train["Modo Principal"].values, y_test["Modo Principal"].values, params_xgb)

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption


In [45]:
explainer = shap.Explainer(reg_xgb)
shap_values = explainer(X_test)

In [47]:
shap.plots.waterfall(shap_values[0])

Exception: waterfall_plot requires a scalar base_values of the model output as the first parameter, but you have passed an array as the first parameter! Try shap.waterfall_plot(explainer.base_values[0], values[0], X[0]) or for multi-output models try shap.waterfall_plot(explainer.base_values[0], values[0][0], X[0]).

In [53]:
shap.waterfall_plot(explainer.expected_value[0], shap_values.values[0][0], X_train[0])

KeyError: 0

In [57]:
shap_values[0][0]

.values =
array([ 1.2253039e-03,  0.0000000e+00,  8.9271978e-02,  2.9317087e-03,
        0.0000000e+00, -8.3737306e-02, -2.2398393e-01,  0.0000000e+00,
       -1.6673954e-02, -1.7613564e-04,  1.2189511e-02], dtype=float32)

.base_values =
array([1.245961  , 0.02645096, 1.0729733 , 0.08447349, 0.59136486,
       0.4691026 , 0.19297528, 0.05571976, 0.02086747, 0.09735176,
       0.81310415], dtype=float32)

.data =
37.0

In [54]:
shap_values.values[0][0]

array([ 1.2253039e-03,  0.0000000e+00,  8.9271978e-02,  2.9317087e-03,
        0.0000000e+00, -8.3737306e-02, -2.2398393e-01,  0.0000000e+00,
       -1.6673954e-02, -1.7613564e-04,  1.2189511e-02], dtype=float32)

In [50]:
dir(explainer)

['_Tree__dynamic_expected_value',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_compute_main_effects',
 '_get_shap_interactions_output',
 '_get_shap_output',
 '_instantiated_load',
 '_validate_inputs',
 'assert_additivity',
 'data',
 'data_missing',
 'expected_value',
 'explain_row',
 'feature_names',
 'feature_perturbation',
 'link',
 'load',
 'masker',
 'model',
 'model_output',
 'output_names',
 'save',
 'shap_interaction_values',
 'shap_values',
 'supports_model_with_masker']

In [29]:
explainer.expected_value

[1.245961,
 0.026450962,
 1.0729733,
 0.08447349,
 0.59136486,
 0.4691026,
 0.19297528,
 0.055719763,
 0.020867467,
 0.09735176,
 0.81310415]

In [37]:
shap_values_unit = explainer.shap_values(X_train.sample(1))

In [41]:
shap_values.data[0]

array([ 4.30000000e+01,  4.00000000e+00,  1.25833333e+01,  1.20000000e+01,
        1.48000000e+02,  2.90349222e+03,  9.42027318e+02,  2.94061385e+03,
        4.81002110e+02,  3.43359600e+06,  5.00000000e+01,  1.20000000e+01,
        1.48000000e+02,  3.05755947e+03,  2.39935003e+03,  3.11712377e+03,
        8.23888393e+02,  2.83823100e+06,  2.10000000e+01,  6.66666667e-01,
        0.00000000e+00,  0.00000000e+00,  2.16666667e+04,  1.50759709e+03,
       -2.98733333e+01,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00])

In [43]:
shap_values.values[0]

array([[ 1.2253039e-03,  0.0000000e+00,  2.0044221e-01,  2.9317087e-03,
         0.0000000e+00, -1.9584884e-01, -2.0646983e-01,  0.0000000e+00,
        -1.7066630e-02, -8.6170141e-05,  3.9781998e-03],
       [ 0.0000000e+00,  0.0000000e+00, -9.2477398e-03,  0.0000000e+00,
         0.0000000e+00,  4.3334737e-03, -3.2642014e-02,  0.0000000e+00,
        -3.3551443e-04, -1.3780817e-03, -2.6316740e-02],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, -1.2609171e-04,
         0.0000000e+00, -1.4903171e-04, -2.0578040e-03,  0.0000000e+00,
        -4.6277677e-05, -4.9194728e-04,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, -1.2693671e-04,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         4.8078102e-04,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00, -5.4143144e-05, -2.9744369e-05, -9.0333575e-05,
         0.0000000e+00,  0.0000000e+00, -4.1509019e-03,  0.0000000e+00,
         2.4881167e-04,  0.0000000e+00,  0.0000000e+

In [42]:
shap.force_plot(explainer.expected_value[0], shap_values.values[0], shap_values.data[0])

IndexError: index 11 is out of bounds for axis 0 with size 11

In [38]:
shap_values_unit

[array([[ 0.0012253 ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        , -0.03683075,
          0.        ,  0.        , -0.00236585, -1.2247565 ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ]], dtype=float32),
 array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         -5.4143144e-05,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00,  0.0000000e+00, -1.7300565e-04,  0.0000000e+00,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00, -1.0105655e-02,
         -4.8787488e-06, -6.3276216e-02,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.00000